In [1]:
# 분야 2 | 유선 네트워크 장비의 신속한 조치를 위한 경보 유형 분류

# 아래의 코드는 다음과 같은 순서로 구성되어 있습니다.
# 1. 데이터 로드 및 전처리
# 2. 모델 학습 및 예측
# 3. 결과 제출
# 본 코드는 결과물 제출까지의 이해를 돕기위한 코드로, 반드시 이 구조를 따를 필요는 없습니다.
# 데이터 전처리, 모델의 구성 등에서 다양한 시도를 하시되, 올바른 채점을 위한 최종 결과물의 형태에 유의하시기 바랍니다.

In [2]:
# 1. 데이터 로드
import numpy as np
import pandas as pd

def ppr_data(q2_train, q2_test):
    # TODO: 데이터 전처리 코드 구현 ---------- #
    # slotna 변수 생성
    q2_train['slotna'] = q2_train['slot'].isna().astype(int)
    q2_test['slotna'] = q2_test['slot'].isna().astype(int)

    # 메시지 전처리
    for i in range(len(q2_train)):
      s = q2_train['alarmmsg_original'].iloc[i]
      s = s.lower()
      s = s.replace('(', '').replace(')', '') # 괄호 없애기
      s = s.replace('-', ' ').replace('_', ' ') # '-', '_' 구분자 공백으로 바꾸기
      s = s.split() # 공백 기준으로 split
      for _ in s: # 조사나 의미 없는 단어 삭제
          if 'of' in s:
              s.remove('of')
          if 'los' in s:
              s.remove('los')
          if 'loss' in s:
              s.remove('loss')
          if 'by' in s:
              s.remove('by')
          if 'with' in s:
              s.remove('with')
          if 'fail' in s:
              s.remove('fail')
          if 'error' in s:
              s.remove('error')
          if 'err' in s:
              s.remove('err')
      q2_train['alarmmsg_original'].iloc[i] = s

    for i in range(len(q2_test)):
      if q2_test['root_cause_domain'].iloc[i,] == 'C':
          input_string = q2_test['alarmmsg_original'].iloc[i,]
          uppercase_letters = [char for char in input_string if char.isupper()]
          result = ''.join(uppercase_letters)

          # result 전처리
          if result == 'LOS' or result == 'LC':
              result = ''

          # 원래 메시지 전처리
          input_string = input_string.lower()
          input_string = input_string.replace('(', '').replace(')', '') # 괄호 없애기
          input_string = input_string.replace('-',' ').replace('_',' ').replace(':', '') # ':' 없애기
          input_string = input_string.split() # 공백 기준으로 split
          for _ in input_string: # 조사나 의미 없는 단어 삭제
              if 'of' in input_string:
                  input_string.remove('of')
              if 'los' in input_string:
                  input_string.remove('los')
              if 'loss' in input_string:
                  input_string.remove('loss')
              if 'by' in input_string:
                  input_string.remove('by')
              if 'with' in input_string:
                  input_string.remove('with')
              if 'fail' in input_string:
                  input_string.remove('fail')
              if 'error' in input_string:
                  input_string.remove('error')
              if 'err' in input_string:
                  input_string.remove('err')
              if 'failure' in input_string:
                  input_string.remove('failure')
          if result:
              q2_test['alarmmsg_original'].iloc[i,] = [input_string, [result.lower()]]
              q2_test['alarmmsg_original'].iloc[i,] = sum(q2_test['alarmmsg_original'].iloc[i,], [])
          else:
              q2_test['alarmmsg_original'].iloc[i,] = input_string

      else:
          input_string1 = q2_test['alarmmsg_original'].iloc[i,]

          # 원래 메시지 전처리
          input_string1 = input_string1.lower()
          input_string1 = input_string1.replace('(', '').replace(')', '') # 괄호 없애기
          input_string1 = input_string1.replace('-',' ').replace('_',' ').replace(':', '') # ':' 없애기
          input_string1 = input_string1.split() # 공백 기준으로 split
          for _ in input_string: # 조사나 의미 없는 단어 삭제
              if 'of' in input_string1:
                  input_string1.remove('of')
              if 'los' in input_string1:
                  input_string1.remove('los')
              if 'loss' in input_string1:
                  input_string1.remove('loss')
              if 'by' in input_string1:
                  input_string1.remove('by')
              if 'with' in input_string1:
                  input_string1.remove('with')
              if 'fail' in input_string1:
                  input_string1.remove('fail')
              if 'error' in input_string1:
                  input_string1.remove('error')
              if 'err' in input_string1:
                  input_string1.remove('err')
              if 'failure' in input_string1:
                  input_string1.remove('failure')
              if 'ether' in input_string1:
                  input_string1 = ['eth']

          q2_test['alarmmsg_original'].iloc[i,] = input_string1

    """
    # alarmno 13자리/9자리 범주형
    for i in range(len(q2_train)):
        if len(str(q2_train['alarmno'][i])) == 9:
            q2_train['alarmno'][i] = 1
        else:
            q2_train['alarmno'][i] = 0
    for i in range(len(q2_test)):
        if len(str(q2_test['alarmno'][i])) == 9:
            q2_test['alarmno'][i] = 1
        else:
            q2_test['alarmno'][i] = 0
    """
    # alarmno 자릿수
    for i in range(len(q2_train)):
        q2_train['alarmno'][i] = int(str(q2_train['alarmno'][i])[0:6])
    for i in range(len(q2_test)):
        q2_test['alarmno'][i] = int(str(q2_test['alarmno'][i])[0:6])
    
    # 필요 없는 변수 drop
    q2_train.drop(labels = ['slot', 'port', 'sva', 'root_cause_domain'], axis = 1, inplace = True)
    q2_test.drop(labels = ['slot', 'port', 'sva', 'root_cause_domain'], axis = 1, inplace = True)

    # 열 순서 정렬
    col = q2_train.columns.to_numpy()
    col = col[[0, 1, 2, 3, 4, 5, 6, 7, 9, 8]]
    q2_train = q2_train[col]

    # time 변수 datetime으로 타입 변경
    q2_train['alarmtime'] = pd.to_datetime(q2_train['alarmtime'])
    q2_test['alarmtime'] = pd.to_datetime(q2_test['alarmtime'])

    # train, test 데이터 합치기
    q2_train['train'] = 1
    q2_test['train'] = 0
    col = q2_train.columns.to_numpy()
    col = col[[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 9]]
    q2_train = q2_train[col]
    data_total = pd.concat([q2_train, q2_test])

    # alarmtime 변수로 지속시간 변수 추가
    data = data_total.groupby('ticketno')['alarmtime'].agg(**{'min_time':'min', 'max_time':'max'}).reset_index()
    data['duration'] = data['max_time'] - data['min_time']
    for i in range(len(data['duration'])):
      data['duration'][i] = data['duration'][i].seconds / 60
    data['duration'] = data['duration'].apply(pd.to_numeric)

    # 전표별 level 당 개수 변수 생성
    level = data_total.groupby(['ticketno','alarmlevel']).size().unstack(fill_value = 0).reset_index()
    data['level3'] = level[3]/(level[3]+level[4]+level[5]+level[7])
    data['level4'] = level[4]/(level[3]+level[4]+level[5]+level[7])
    data['level5'] = level[5]/(level[3]+level[4]+level[5]+level[7])
    data['level7'] = level[7]/(level[3]+level[4]+level[5]+level[7])
    data.drop(labels=['max_time'], axis = 1, inplace = True)

    # 전표별 alarmmsg 이어붙이기
    message = data_total.groupby('ticketno')['alarmmsg_original'].apply(list).reset_index()
    for i in range(len(message)):
      message['alarmmsg_original'].iloc[i] = sum(message['alarmmsg_original'].iloc[i],[])

    data = pd.merge(data, message)
    data = pd.merge(data, data_total.groupby('ticketno')['train'].agg(**{'train':max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['root_cause_type'].agg(**{'type':max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['slotna'].agg(**{'slotna': max}).reset_index())   
    data = pd.merge(data, data_total.groupby('ticketno')['alarmno'].agg(**{'alarmno': max}).reset_index())

    col = data.columns.to_numpy()
    col = col[[0, 1, 2, 11, 3, 4, 5, 6, 7, 8, 10, 9]]
    data = data[col]

    # data 시간별로 sort
    data.sort_values('min_time', inplace = True)

    # train, test 분리
    data_train = data[data['train'] == 1]
    data_test = data[data['train'] == 0]
    data_train.drop(labels = ['train'], axis = 1, inplace = True)
    data_test.drop(labels = ['train'], axis = 1, inplace = True)

    # x, y 변수 분리
    x_train_df = data_train.iloc[:, :-1]
    y_train_df = data_train['type']
    x_test_df = data_test.iloc[:, :-1]

    # ------------------------------------- #
    return x_train_df, y_train_df, x_test_df


train_df = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/Q2_train.csv')
test_df = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/Q2_test.csv')

x_train_df, y_train_df, x_test_df = ppr_data(train_df, test_df)

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21464\2366421433.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_train['alarmmsg_original'].iloc[i] = s
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21464\2366421433.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_test['alarmmsg_original'].iloc[i,] = input_string
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_21464\2366421433.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [3]:
x_train_df

,ticketno,min_time,duration,alarmno,level3,level4,level5,level7,alarmmsg_original,slotna
714,21122633.0,2022-12-01 00:02:24+09:00,1424.933333,166999,0.0,0.0,1.0,0.0,"[eth, eth, eth, eth, eth, eth, eth, eth, eth, ...",0
718,21123591.0,2022-12-01 01:08:00+09:00,0.333333,166982,0.0,0.0,0.0,1.0,"[opt, opt]",0
716,21123581.0,2022-12-01 01:08:00+09:00,0.000000,166982,0.0,0.0,0.5,0.5,"[opt, dcc]",0
717,21123584.0,2022-12-01 01:08:05+09:00,0.000000,166982,0.0,0.0,1.0,0.0,"[lsp, loc]",0
715,21123580.0,2022-12-01 01:08:05+09:00,0.000000,166982,0.0,0.0,1.0,0.0,"[lsp, loc]",0
...,...,...,...,...,...,...,...,...,...,...
704,20975823.0,2022-12-24 17:05:37+09:00,0.000000,166927,0.0,1.0,0.0,0.0,[psu],1
250,17909854.0,2022-12-24 17:48:01+09:00,0.000000,910185,0.0,1.0,0.0,0.0,[psu],1
251,17909870.0,2022-12-24 17:48:22+09:00,0.000000,910185,0.0,1.0,0.0,0.0,[psu],1
1353,21809789.0,2022-12-24 22:27:42+09:00,0.533333,167197,0.0,0.0,0.0,1.0,"[opt, opt]",0


In [105]:
# 2. 모델 학습 및 예측
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

class MyModel:
    def __init__(self) -> None:
        self.model = None

    def train(self, x_train, y_train):
        # TODO: 모델 학습 코드 구현 ---------- #
        # TfidfVertorizer로 메시지 수치화 후 Multinomial naive bayes 모델링

        # Create transformers
        message_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False, min_df=4, max_df=0.8)
        additional_scaler = MinMaxScaler()

        # Define column indices for the ColumnTransformer
        message_col_idx = 'alarmmsg_original'
        additional_cols = ['duration', 'alarmno', 'level3', 'level4', 'level5', 'level7', 'slotna']

        # Create the ColumnTransformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('message', message_vectorizer, message_col_idx),
                ('additional', additional_scaler, additional_cols)
            ],
            remainder='passthrough'
        )

        # Create individual classifiers
        classifier1 = MultinomialNB(alpha=0.7)
        classifier2 = CatBoostClassifier(n_estimators=100)  # Adjust hyperparameters

        # Create the VotingClassifier
        voting_classifier = VotingClassifier(
            estimators=[
                ('nb', classifier1),
                ('cb', classifier2)
            ],
            voting='soft'  # You can use 'hard' or 'soft' voting
        )

        # Create a pipeline with the ColumnTransformer and the classifier
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', voting_classifier)
        ])

        x_train = x_train[['alarmmsg_original'] + additional_cols]

        # Train the pipeline
        self.model = pipeline.fit(x_train, y_train)


        # --------------------------------- #
        pass

    def predict(self, x_test):
        # 1. ticketno 컬럼은 입력받은 값으로 채우고,
        # 2. pred 컬럼은 모두 'LinkCut' 값으로 채운 데이터프레임 생성
        pred_df = pd.DataFrame({'ticketno': x_test['ticketno'].values, 'root_cause_type': self.model.predict(x_test)})
        pred_df.sort_values('ticketno', inplace=True)
        pred_df.reset_index(drop=True, inplace=True)
        return pred_df

model = MyModel()
model.train(x_train_df, y_train_df)
y_pred = model.predict(x_test_df) # 0.91657

c:\Anaconda3\envs\venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Learning rate set to 0.5
0:	learn: 0.4797550	total: 3.65ms	remaining: 362ms
1:	learn: 0.2985743	total: 7.74ms	remaining: 380ms
2:	learn: 0.1978202	total: 11ms	remaining: 355ms
3:	learn: 0.1359964	total: 13.9ms	remaining: 333ms
4:	learn: 0.0985810	total: 16.9ms	remaining: 321ms
5:	learn: 0.0745337	total: 19.6ms	remaining: 307ms
6:	learn: 0.0587138	total: 23.3ms	remaining: 310ms
7:	learn: 0.0516019	total: 26.4ms	remaining: 303ms
8:	learn: 0.0484487	total: 29.2ms	remaining: 296ms
9:	learn: 0.0441446	total: 32.4ms	remaining: 292ms
10:	learn: 0.0411229	total: 34.8ms	remaining: 282ms
11:	learn: 0.0376022	total: 37.7ms	remaining: 276ms
12:	learn: 0.0352154	total: 40.2ms	remaining: 269ms
13:	learn: 0.0314462	total: 43.1ms	remaining: 265ms
14:	learn: 0.0279672	total: 46ms	remaining: 261ms
15:	learn: 0.0258610	total: 49.1ms	remaining: 258ms
16:	learn: 0.0250520	total: 52.9ms	remaining: 258ms
17:	learn: 0.0231327	total: 56.2ms	remaining: 256ms
18:	learn: 0.0222003	total: 58.8ms	remaining: 251ms
1

In [106]:
print(list(y_pred['root_cause_type']).count('LinkCut'))
print(list(y_pred['root_cause_type']).count('PowerFail'))
print(list(y_pred['root_cause_type']).count('UnitFail'))

2580
1645
102


In [5]:
# 3. 결과 제출
# 본 코드는 제출되는 파일의 형태에 대한 가이드로, 반드시 아래 구조를 따를 필요 없이 자유롭게 코드를 작성해도 무방합니다.
# 제출 포맷에 대해선 data/Q2_label_sample.csv를 참조하세요.
#
# 분야 2의 경우, 전표(ticket) 하나에 하나의 근원장애(root_cause_type)을 매칭해야 합니다.
#   주의: 경보(alarm) 개수와 전표(ticket) 개수는 다르며, 예측할 대상은 전표입니다.
#   주의: ticketno 컬럼 기준으로 오름차순 정렬이 필요합니다.
# 분야 2의 제출 파일은 2개 컬럼 [ticketno, root_cause_type]을 가져야 합니다.

def submitResult(pred):
    try:
        label = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/Q2_label_sample.csv')
        # ticketno 순서와 개수가 일치하는지 확인
        if (label['ticketno'] == pred['ticketno']).all():
            print("Check: ticketno 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 ticketno가 일치하지 않습니다.")
            return

        pred.to_csv('Q2_submitResult.csv', index=False)
        print("Done : Q2_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

submitResult(y_pred)

Check: ticketno 순서와 샘플 수가 일치합니다.
Done : Q2_submitResult.csv 파일로 저장되었습니다.
